- `SelfAttention_v1`

In [1]:
import torch.nn as nn
import torch 

class SelfAttention_v1(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()      # Khởi tạo lớp cha nn.Module
        self.dim_out = dim_out   # Kích thước đầu ra
        # rand - phân phối đều liên tục trong khoảng [0, 1)
        self.W_query = nn.Parameter(torch.rand(dim_in, dim_out))  # Ma trận trọng số cho query
        self.W_key = nn.Parameter(torch.rand(dim_in, dim_out))    # Ma trận trọng số cho key
        self.W_value = nn.Parameter(torch.rand(dim_in, dim_out))  # Ma trận trọng số cho value

    def forward(self, x):
        queries = x @ self.W_query # Tính queries
        keys = x @ self.W_key      # Tính keys
        values = x @ self.W_value  # Tính values

        attn_scores = queries @ keys.T      # Tính attention scores (omega)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vectors = attn_weights @ values  # Tính context vectors (z)

        return context_vectors

In [2]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)

In [3]:
torch.manual_seed(123)
self_attention_v1 = SelfAttention_v1(dim_in=3, dim_out=2)   
print(self_attention_v1(inputs))    # run forward with inputs

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


- `SelfAttention_v2`: use _nn.Linear_ 

In [ ]:
class SelfAttention_v2(nn.Module):
    def __init__(self, dim_in, dim_out, qkv_bias=False):
        super().__init__()     # Khởi tạo lớp cha nn.Module
        self.dim_out = dim_out  # Kích thước đầu ra
        self.W_query = nn.Linear(dim_in, dim_out, bias=qkv_bias)  # Lớp Linear cho query
        self.W_key = nn.Linear(dim_in, dim_out, bias=qkv_bias)    # Lớp Linear cho key
        self.W_value = nn.Linear(dim_in, dim_out, bias=qkv_bias)  # Lớp Linear cho value

    def forward(self, x):
        queries = self.W_query(x)   # X . Wq 
        keys = self.W_key(x)         # X . Wk
        values = self.W_value(x)     # X . Wv

        attn_scores = queries @ keys.T      # Tính attention scores (omega)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vectors = attn_weights @ values  # Tính context vectors (z)

        return context_vectors

In [6]:
torch.manual_seed(123)
self_attention_v2 = SelfAttention_v2(dim_in=3, dim_out=2, qkv_bias=False)
print(self_attention_v2(inputs))    # run forward with inputs

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)


- 2 phiên bản `SelfAttention` cho ra 2 output khác nhau, vì weights khởi tạo của chúng khác nhau.

### So sánh `SelfAttention_v1` và `SelfAttention_v2`

- Ở đây chúng ta sẽ chuyển `weights` khởi tạo từ v2 qua v1 để thể hiện sự so sánh.

In [8]:
layer = nn.Linear(3, 2, bias=False)
layer.weight.shape

torch.Size([2, 3])

- Vì `nn.Linear` lưu ma trận weights ở dạng _chuyển vị (transposed)_ `weight.T`.

- Nếu `linear.weight` có kích thước `[out_dim, in_dim]` thì `SelfAttention.W_query` có kích thước `[in_dim, out_dim]`.

- Vì vậy khi gán cần chuyển vị ma trận.

In [9]:
self_attention_v1 = SelfAttention_v1(dim_in=3, dim_out=2)
self_attention_v2 = SelfAttention_v2(dim_in=3, dim_out=2, qkv_bias=False)

self_attention_v1.W_query.data = self_attention_v2.W_query.weight.T.data.clone()
self_attention_v1.W_key.data = self_attention_v2.W_key.weight.T.data.clone()
self_attention_v1.W_value.data = self_attention_v2.W_value.weight.T.data.clone()    

output_v1 = self_attention_v1(inputs)
output_v2 = self_attention_v2(inputs)
print("Output v1:\n", output_v1)
print("Output v2:\n", output_v2)

Output v1:
 tensor([[ 0.0309, -0.2902],
        [ 0.0308, -0.2899],
        [ 0.0308, -0.2898],
        [ 0.0300, -0.2897],
        [ 0.0292, -0.2891],
        [ 0.0306, -0.2900]], grad_fn=<MmBackward0>)
Output v2:
 tensor([[ 0.0309, -0.2902],
        [ 0.0308, -0.2899],
        [ 0.0308, -0.2898],
        [ 0.0300, -0.2897],
        [ 0.0292, -0.2891],
        [ 0.0306, -0.2900]], grad_fn=<MmBackward0>)


- Từ đó thấy được cả 2 version đều giống nhau về mặt chức năng.